In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from pivottablejs import pivot_ui
import pivottablejs
import qgrid
import matplotlib.pyplot as plt

In [2]:
path='C:\\Users\\camendol\\Desktop\\'
file='PCRs.xlsx'

file_path=path+file

In [3]:
#Open connection to EXCEL
xl_file = pd.ExcelFile(file_path)
xl_file.sheet_names 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\camendol\\Desktop\\PCRs.xlsx'

In [ ]:
#Read a tab
pcr_data=xl_file.parse('PCRs')

In [4]:
#Apply Date typing
pcr_data['Created']=pd.to_datetime(pcr_data['Creation Date'].str[0:20])
pcr_data['Created_yr_mon'] = pd.to_datetime(pcr_data['Created'].dt.date).map(lambda dt: dt.replace(day=1))
pcr_data['Created_yr'] = pd.to_datetime(pcr_data['Created'].dt.date).map(lambda dt: dt.replace(day=1,month=1))
pcr_data['Updated']=pd.to_datetime(pcr_data['Last Update Date'].str[0:20])
pcr_data['Duration']=pcr_data['Updated'] - pcr_data['Created']
pcr_data['Duration']=pcr_data['Duration']/np.timedelta64(1,'D')

pcr_data['QA Expected']=pd.to_datetime(pcr_data['Expected Date to QA'].str[0:20])
pcr_data['QA Delivered']=pd.to_datetime(pcr_data['Actual Date to QA'].str[0:20])
pcr_data['PCR Approved']=pd.to_datetime(pcr_data['New PCR Approval Date'].str[0:20]) 

pcr_data['Update_Elapsed']=(dt.datetime.today() - pcr_data['Updated']).dt.days 
#Maybe bin off day buckets
bin_30s = [-1,15,30,45, 60, 75 ,90, 120, 150, 180, 210, 240, 270, 300, 330, 360, np.inf]
bin_labels = ['15','30', '45','60','75' ,'90', '120', '150','180','210','240','270','300','330','360','361+']
pcr_data['Updated_Seg']=pd.cut(pcr_data['Update_Elapsed'],bin_30s,labels=bin_labels)
pcr_data['Duration_Seg']=pd.cut(pcr_data['Duration'],bin_30s,labels=bin_labels)
#pcr_data.loc[pcr_data['Updated_Seg'].isna()]
#Week Bins
week_bin = [-1,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133,140,147,154,161,168,175,182,np.inf]
week_labels = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27+']
pcr_data['Updated_WKs']=pd.cut(pcr_data['Update_Elapsed'],week_bin,labels=week_labels)

NameError: name 'pcr_data' is not defined

In [ ]:
current_team=[ 'Lois Nordstrom'\
              , 'Sue Szubzda'\
              ,'Tricia Huebner'\
              , 'Padmaja Gandhi'\
              ,'Phanikumar Jetti'\
              , 'Pragada Kumar'\
              , 'Christopher Amendola'\
              ,''\
              ,' ']
old_team=[ 'Lois Nordstrom'\
              , 'Sue Szubzda'\
              ,'Tricia Huebner'\
              , 'Padmaja Gandhi'\
              ,'Phanikumar Jetti'\
              , 'Pragada Kumar'\
              , 'Christopher Amendola'\
              , 'Santhi George'\
              ,''\
              ,' ']

In [ ]:
#Created:Year-to-date
created_YTD=pcr_data.loc[ ((pcr_data['Owner'].isin(old_team) | pcr_data['Owner'].isnull())
                               & (pcr_data['Created']>'2018-12-31'))\
                               ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]\
        .sort_values(by='Created')

In [ ]:
completed_20=created_YTD.loc[(  (created_YTD['Impact Kanban State']=='Released')\
                             | (created_YTD['Impact Kanban State']=='UAT'))\
                             & (created_YTD['Created_yr']=='2020-01-01')                            
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]

In [ ]:
active_20=created_YTD.loc[(  (created_YTD['Impact Kanban State']=='Dev') \
                          | (created_YTD['Impact Kanban State']=='Approval')\
                          | (created_YTD['Impact Kanban State']=='Requests'))\
                          & (created_YTD['Created_yr']=='2020-01-01')
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]

11.07.19: 
    Completed Requests 2019: 79
    Active Requests 2019: 28
    
11.14.19:
    Completed Requests 2019: 80
    Active Requests 2019: 27
    
11.17.19:
    Completed Requests 2019: 80
    Active Requests 2019: 29
    
11.19.19:
    Completed Requests 2019: 82
    Active Requests 2019: 26
  
11.20.19
    Completed Requests 2019: 82
    Active Requests 2019: 27
    
11.23.19
    Completed Requests 2019: 84
    Active Requests 2019: 24
    
11.26.19
    Completed Requests 2019: 85
    Active Requests 2019: 25
    
11.27.19
    Completed Requests 2019: 85
    Active Requests 2019: 26

12.05.19
    Completed Requests 2019: 85
    Active Requests 2019: 27
    
12.06.19
    Completed Requests 2019: 85
    Active Requests 2019: 29
    
12.10.19
   Completed Requests 2019: 90
   Active Requests 2019: 27

12.11.19
   Completed Requests 2019: 91
   Active Requests 2019: 27
   
12.12.19
   Completed Requests 2019: 92
   Active Requests 2019: 26
   
12.16.19
   Completed Requests 2019: 94
   Active Requests 2019: 26
   
12.17.19
   Completed Requests 2019: 95
   Active Requests 2019: 27
   
12.19.19
   Completed Requests 2019: 96
   Active Requests 2019: 26

In [ ]:
print('Completed Requests 2020: '+str(len(completed_20)))
print('Active Requests 2020: '+str(len(active_20)))

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 5]

In [ ]:
pivot_ui(completed_20,cols=['Duration_Seg'],rendererName='Bar Chart')

In [ ]:
pivot_ui(active_20,cols=['Updated_WKs'],rendererName='Bar Chart')

In [ ]:
pivot_ui(created_YTD,rows=['Created_yr'],cols=['Created_yr_mon'],rendererName='Bar Chart')

In [ ]:
pivot_ui(created_YTD)

In [ ]:
qgrid.show_grid(active_20[[ 'Name'\
                          ,'Owner'\
                          ,'Updated_WKs'\
                          ,'Duration'\
                          ,'Impact Kanban State'\
                          ]],show_toolbar=True)

In [ ]:
#Create Excel sheets for each person
# for whom in active_19['Owner'].unique():
#     excel_book='C:\\Users\\camendol\\Documents\\PCRs\\'+whom+'.xlsx'
#     current=active_19.loc[active_19['Owner']==whom,('Formatted ID','Name','Duration_Seg','Updated_WKs','Impact Kanban State')]
#     print(excel_book)
#     with pd.ExcelWriter(excel_book) as writer:
#         pd.DataFrame(current)\
#         .to_excel(writer, sheet_name='User Stories')